## 处理数据

In [3]:
from datasets import load_dataset

raw_data_path="../data/bfcl_multi_turn.json"
raw_ds=load_dataset("json",data_files=raw_data_path,split="train")


/dfs/data/uv-venv/modelscope/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
with open("../data/system_prompt.txt","w") as f:
    f.write(
        raw_ds[0]["0"]["content"].split("[{\"name\"")[0]
    )

In [5]:
def convert_to_openai(example):
    """
    将 HardGen 格式的单条样本转换为 OpenAI messages 格式
    """
    messages = []
    
    # --- A. 处理 System Prompt (Key "0") ---
    # dataset 加载后，key 对应的值如果是 None 表示该样本没有此轮（因为是稀疏列）
    if example.get("0") is not None:
        system_data = example["0"]
        system_content = system_data.get("content", "")
        
        # System 包含 content (及 tools 定义)
        messages.append({
            "role": "system",
            "content": system_content
        })

    # --- B. 处理对话轮次 (Key "1", "2", "3"...) ---
    # 获取所有是数字的 key，排除 "0"
    all_keys = list(example.keys())
    turn_keys = [k for k in all_keys if k.isdigit() and k != "0"]
    
    # 必须按数字大小排序，保证对话顺序
    sorted_keys = sorted(turn_keys, key=int)
    
    for key in sorted_keys:
        turn_data = example[key]
        
        # 因为 dataset 是稀疏的，某些样本可能没有 key "10"，值为 None
        if turn_data is None:
            continue
            
        role = turn_data.get("role")
        content = turn_data.get("content")
        
        # --- C. 映射逻辑 ---
        if role == "user":
            messages.append({"role": "user", "content": content})
            
        elif role == "assistant":
            # 保留 <think> 和 function call
            messages.append({"role": "assistant", "content": content})
            
        elif role == "tool":
            messages.append({"role": "tool", "content": content})

    # 返回新的格式
    return {"messages": messages}

In [7]:
converted_ds = raw_ds.map(
    convert_to_openai,
    remove_columns=raw_ds.column_names
)

Map: 100%|█████████████████████████████████████████████████████████████████████████████| 16978/16978 [00:10<00:00, 1580.38 examples/s]


In [8]:
converted_ds[0]

{'messages': [{'content': 'You are an expert in composing functions. You are given a question and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to achieve the purpose.\nIf none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out.\nYou should only return the function calls in your response.\n\nIf you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\nYou SHOULD NOT include any other text in the response.\n\nAt each turn, you should try your best to complete the tasks requested by the user within the current turn. Continue to output functions to call until you have fulfilled the user\'s request to the best of your ability. Once you have no more functions to call, the system will consider the current turn complete and proceed to the next 

## 加载huggingface ecosystem